In [2]:
import pandas as pd
import numpy as np
import seaborn as sns

import requests
import json
import geopy

import time
import parquet


In [3]:
data = pd.read_csv('despesas-entre-2003-e-2022-UTF-8.csv',sep=';', dtype='str')

In [4]:
data

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC
0,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÍSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N
1,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÍSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N
2,02/01/2003,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÍSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N
3,03/01/2003,***.004.131-**,00592717000170,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIÇOS DE TERCEIROS-PESSOA JURÍDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N
4,03/01/2003,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COMÉRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇÃO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N
...,...,...,...,...,...,...,...,...
113335,03/12/2022,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113336,03/12/2022,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113337,04/12/2022,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO,"518,07",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N
113338,04/12/2022,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N


In [5]:
def get_company_info(cnpj, use_safe_access=True):
    url = f"https://receitaws.com.br/v1/cnpj/{cnpj}"
    querystring = {"token":"XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX","cnpj":"06990590000123","plugin":"RF"}
    if use_safe_access:
        time.sleep(20)
    response = requests.request("GET", url, params=querystring)
    if use_safe_access and not response.ok:
        time.sleep(60)
        response = requests.request("GET", url, params=querystring)
    return json.loads(response.text)

def get_company_adress(info):
    return f"{info['logradouro']}, {info['numero']}, {info['municipio']}, {info['uf']}"

In [6]:
GOOGLE_CNPJ = "03696869000100"
info = get_company_info(GOOGLE_CNPJ, use_safe_access=False)
print(info)
#atividade_principal

#tipo

#logradouro
#número
#bairro
#municipio
#uf
#cep

#natureza_juridica


{'atividade_principal': [{'code': '47.11-3-02', 'text': 'Comércio varejista de mercadorias em geral, com predominância de produtos alimentícios - supermercados'}], 'data_situacao': '03/11/2005', 'complemento': 'BLOCO A', 'tipo': 'MATRIZ', 'nome': 'BIG TRANS COMERCIAL DE ALIMENTOS S/A', 'abertura': '04/11/1999', 'telefone': '(61) 3265-191', 'qsa': [{'nome': 'JORGE HELOU FILHO', 'qual': '10-Diretor'}, {'nome': 'MARIO HABKA', 'qual': '10-Diretor'}], 'situacao': 'ATIVA', 'bairro': 'ASA NORTE', 'logradouro': 'SHCN EQ 402/403', 'numero': 'SN', 'cep': '70.834-400', 'municipio': 'BRASILIA', 'uf': 'DF', 'porte': 'DEMAIS', 'natureza_juridica': '205-4 - Sociedade Anônima Fechada', 'fantasia': 'BIG BOX SUPERMERCADOS', 'cnpj': '03.696.869/0001-00', 'ultima_atualizacao': '2022-12-27T17:17:28.815Z', 'status': 'OK', 'email': '', 'efr': '', 'motivo_situacao': '', 'situacao_especial': '', 'data_situacao_especial': '', 'atividades_secundarias': [{'code': '00.00-0-00', 'text': 'Não informada'}], 'capital_

In [7]:
data['ATIVIDADE_PRINCIPAL'] = ''
data['MATRIZ'] = ''
data['NATUREZA_JURIDICA'] = ''
data['LOGRADOURO'] = ''
data['NUMERO'] = ''
data['COMPLEMENTO'] = ''
data['BAIRRO'] = ''
data['MUNICIPIO'] = ''
data['UF'] = ''
data['CEP'] = ''

In [8]:
data

,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ATIVIDADE_PRINCIPAL,MATRIZ,NATUREZA_JURIDICA,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
0,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÍSTICA,"R$ 1.696,90",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,,,,,,,,,,
1,02/01/2003,***.010.008-**,31349202000177,JW TRANSPORTADORA TURÍSTICA,"R$ 1.336,60",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.220539/2003-63.R.08.18/12/2002.31/12/2014.N,,,,,,,,,,
2,02/01/2003,***.868.251-**,31349202000177,JW TRANSPORTADORA TURÍSTICA,"R$ 1.396,43",D,LOCACAO DE MEIOS DE TRANSPORTE,00200.600539/2003-63.R.08.18/12/2002.31/12/2014.N,,,,,,,,,,
3,03/01/2003,***.004.131-**,00592717000170,FLORES ALVORADA,"R$ 8.585,00",C,OUTROS SERVIÇOS DE TERCEIROS-PESSOA JURÍDICA,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,,,,,,,,,,
4,03/01/2003,***.004.131-**,00839308000125,CRISTAL LIMPEZA - COMÉRCIO DE MATERIAIS PARA L...,"R$ 251,50",C,MATERIAL DE LIMPEZA E PROD. DE HIGIENIZAÇÃO,00200.800212/2003-91.R.08.07/12/2002.31/12/2014.N,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113335,03/12/2022,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
113336,03/12/2022,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
113337,04/12/2022,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO,"518,07",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
113338,04/12/2022,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,


In [9]:
data['DATA PGTO'] = pd.to_datetime(data['DATA PGTO'])
data = data[data['DATA PGTO'].apply(lambda x: x.year)>=2018]
data


C:\Users\gusta\AppData\Local\Temp\ipykernel_3976\2768678086.py:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data['DATA PGTO'] = pd.to_datetime(data['DATA PGTO'])


,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ATIVIDADE_PRINCIPAL,MATRIZ,NATUREZA_JURIDICA,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
96625,2018-01-01,***.118.204-**,37064730000166,5 ESTRELAS,"R$ 20,87",C,GENEROS DE ALIMENTAÇÃO,00140.000731/2017-15.R.08.09/01/2018-31/12/2018.N,,,,,,,,,,
96626,2018-02-01,***.024.754-**,00394502017209,CENTRO DE ADESTRAMENTO DA MARAMBAIA (CADIM),"R$ 5.220,00",D,HOSPEDAGENS,00150.000691/2017-84.R.08.26/12/2017.31/12/2018.N,,,,,,,,,,
96627,2018-02-01,***.002.851-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 545,06",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,,,,,,,,,,
96628,2018-02-01,***.002.851-**,38030169000167,SUPERMERCADO VAREJAO OBA LTDA,"R$ 204,19",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,,,,,,,,,,
96629,2018-02-01,***.002.851-**,47508411103559,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO PÃO DE AÇ...,"R$ 203,20",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113335,2022-03-12,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
113336,2022-03-12,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
113337,2022-04-12,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO,"518,07",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
113338,2022-04-12,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,


In [10]:
data = data.reset_index()
data

,index,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ATIVIDADE_PRINCIPAL,MATRIZ,NATUREZA_JURIDICA,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP
0,96625,2018-01-01,***.118.204-**,37064730000166,5 ESTRELAS,"R$ 20,87",C,GENEROS DE ALIMENTAÇÃO,00140.000731/2017-15.R.08.09/01/2018-31/12/2018.N,,,,,,,,,,
1,96626,2018-02-01,***.024.754-**,00394502017209,CENTRO DE ADESTRAMENTO DA MARAMBAIA (CADIM),"R$ 5.220,00",D,HOSPEDAGENS,00150.000691/2017-84.R.08.26/12/2017.31/12/2018.N,,,,,,,,,,
2,96627,2018-02-01,***.002.851-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 545,06",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,,,,,,,,,,
3,96628,2018-02-01,***.002.851-**,38030169000167,SUPERMERCADO VAREJAO OBA LTDA,"R$ 204,19",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,,,,,,,,,,
4,96629,2018-02-01,***.002.851-**,47508411103559,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO PÃO DE AÇ...,"R$ 203,20",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16710,113335,2022-03-12,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
16711,113336,2022-03-12,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
16712,113337,2022-04-12,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO,"518,07",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,
16713,113338,2022-04-12,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,,,,,,,,,,


In [11]:
info_dict = {}
line = 0
while line < 16715:
    cnpj = data['CPF/CNPJ FORNECEDOR'].iloc[line]
    info = None
    if cnpj in info_dict:
        info = info_dict[cnpj]
    else:
        try:
            info = get_company_info(cnpj)
            info_dict[cnpj] = info
        except:
            print(f"Erro: linha {line}")
    try:
        data.loc[line,'ATIVIDADE_PRINCIPAL'] = info['atividade_principal'][0]['text']
        data.loc[line,'MATRIZ'] = info['tipo']
        data.loc[line,'NATUREZA_JURIDICA'] = info['natureza_juridica']
        data.loc[line,'LOGRADOURO'] = info['logradouro'] 
        data.loc[line,'NUMERO'] = info['numero'] 
        data.loc[line,'COMPLEMENTO'] = info['complemento'] 
        data.loc[line,'BAIRRO'] = info['bairro'] 
        data.loc[line,'MUNICIPIO'] = info['municipio'] 
        data.loc[line,'UF'] = info['uf'] 
        data.loc[line,'CEP'] = info['cep'] 
        data.loc[line,'ENDEREÇO'] = get_company_adress(info)
    except:
        print(f"Erro: cnpj {cnpj} inválido")
    if line%10 == 0:
        print(f"Linha: {line} | CNPJs registrados: {len(info_dict)}")
        data.to_parquet('Dados_processados_metade.parquet')
    line += 1
    

data

Linha: 0 | CNPJs registrados: 1
Linha: 10 | CNPJs registrados: 10
Linha: 20 | CNPJs registrados: 14
Linha: 30 | CNPJs registrados: 19
Linha: 40 | CNPJs registrados: 24
Linha: 50 | CNPJs registrados: 25
Linha: 60 | CNPJs registrados: 30
Linha: 70 | CNPJs registrados: 31
Linha: 80 | CNPJs registrados: 38
Erro: cnpj nan inválido
Linha: 90 | CNPJs registrados: 45
Erro: cnpj nan inválido
Erro: cnpj nan inválido
Linha: 100 | CNPJs registrados: 46
Linha: 110 | CNPJs registrados: 48
Linha: 120 | CNPJs registrados: 54
Linha: 130 | CNPJs registrados: 54
Linha: 140 | CNPJs registrados: 56
Linha: 150 | CNPJs registrados: 58
Erro: cnpj nan inválido
Erro: cnpj nan inválido
Erro: cnpj nan inválido
Erro: cnpj nan inválido
Erro: cnpj nan inválido
Linha: 160 | CNPJs registrados: 61
Linha: 170 | CNPJs registrados: 62
Erro: cnpj nan inválido
Erro: cnpj nan inválido
Linha: 180 | CNPJs registrados: 65
Linha: 190 | CNPJs registrados: 67
Linha: 200 | CNPJs registrados: 68
Linha: 210 | CNPJs registrados: 73
Li

,index,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ATIVIDADE_PRINCIPAL,MATRIZ,NATUREZA_JURIDICA,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP,ENDEREÇO
0,96625,2018-01-01,***.118.204-**,37064730000166,5 ESTRELAS,"R$ 20,87",C,GENEROS DE ALIMENTAÇÃO,00140.000731/2017-15.R.08.09/01/2018-31/12/2018.N,"Lanchonetes, casas de chá, de sucos e similares",MATRIZ,206-2 - Sociedade Empresária Limitada,Q QI 4 BLOCO B,S/N,LOTE 11/37 BLOCO B LOJA 9 COMERCIO LOCAL,GUARA I,BRASILIA,DF,71.010-622,"Q QI 4 BLOCO B, S/N, BRASILIA, DF"
1,96626,2018-02-01,***.024.754-**,00394502017209,CENTRO DE ADESTRAMENTO DA MARAMBAIA (CADIM),"R$ 5.220,00",D,HOSPEDAGENS,00150.000691/2017-84.R.08.26/12/2017.31/12/2018.N,Defesa,FILIAL,101-5 - Órgão Público do Poder Executivo Federal,AV RAFAEL LEVY MIRANDA,S/N,,ITACURUCA,MANGARATIBA,RJ,23.860-000,"AV RAFAEL LEVY MIRANDA, S/N, MANGARATIBA, RJ"
2,96627,2018-02-01,***.002.851-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 545,06",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,"Comércio varejista de mercadorias em geral, co...",MATRIZ,205-4 - Sociedade Anônima Fechada,SHCN EQ 402/403,SN,BLOCO A,ASA NORTE,BRASILIA,DF,70.834-400,"SHCN EQ 402/403, SN, BRASILIA, DF"
3,96628,2018-02-01,***.002.851-**,38030169000167,SUPERMERCADO VAREJAO OBA LTDA,"R$ 204,19",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,********,MATRIZ,206-2 - Sociedade Empresária Limitada,SCLN QUADRA 306 BLOCO D,S/N,LOJA 79,ASA NORTE,BRASILIA,DF,70.310-500,"SCLN QUADRA 306 BLOCO D, S/N, BRASILIA, DF"
4,96629,2018-02-01,***.002.851-**,47508411103559,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO PÃO DE AÇ...,"R$ 203,20",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,"Comércio varejista de mercadorias em geral, co...",FILIAL,204-6 - Sociedade Anônima Aberta,SHCN EQ 508/509,S/N,BLOCO A,PLANO PILOTO,BRASILIA,DF,70.310-500,"SHCN EQ 508/509, S/N, BRASILIA, DF"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16710,113335,2022-03-12,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,Restaurantes e similares,FILIAL,205-4 - Sociedade Anônima Fechada,Q SCES TRECHO 2,CJ 32,: LOJAS A12A4 E 2A5; : PISO NACOES;,ASA SUL,BRASILIA,DF,70.200-002,"Q SCES TRECHO 2, CJ 32, BRASILIA, DF"
16711,113336,2022-03-12,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,Comércio varejista de produtos alimentícios em...,MATRIZ,206-2 - Sociedade Empresária Limitada,ST SHCN CL QUADRA 105 BLOCO D LOJA,54,,ASA NORTE,BRASILIA,DF,70.734-540,"ST SHCN CL QUADRA 105 BLOCO D LOJA, 54, BRASIL..."
16712,113337,2022-04-12,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO,"518,07",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,"Comércio varejista de mercadorias em geral, co...",FILIAL,204-6 - Sociedade Anônima Aberta,SHCN EQ 404/405 BL A,SN,,ASA NORTE,BRASILIA,DF,70.310-500,"SHCN EQ 404/405 BL A, SN, BRASILIA, DF"
16713,113338,2022-04-12,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,Restaurantes e similares,MATRIZ,206-2 - Sociedade Empresária Limitada,ST SHI/S QL 10 LT 22,22,PT SUL,LAGO SUL,BRASILIA,DF,71.630-015,"ST SHI/S QL 10 LT 22, 22, BRASILIA, DF"


In [16]:
data.to_parquet('Dados_processados_COMPLETO.parquet')

In [13]:
len(data['CPF/CNPJ FORNECEDOR'].unique())

2113

In [17]:
pd.read_parquet('Dados_processados_COMPLETO.parquet')

,index,DATA PGTO,CPF SERVIDOR,CPF/CNPJ FORNECEDOR,NOME FORNECEDOR,VALOR,TIPO,SUBELEMENTO DE DESPESA,CDIC,ATIVIDADE_PRINCIPAL,MATRIZ,NATUREZA_JURIDICA,LOGRADOURO,NUMERO,COMPLEMENTO,BAIRRO,MUNICIPIO,UF,CEP,ENDEREÇO
0,96625,2018-01-01,***.118.204-**,37064730000166,5 ESTRELAS,"R$ 20,87",C,GENEROS DE ALIMENTAÇÃO,00140.000731/2017-15.R.08.09/01/2018-31/12/2018.N,"Lanchonetes, casas de chá, de sucos e similares",MATRIZ,206-2 - Sociedade Empresária Limitada,Q QI 4 BLOCO B,S/N,LOTE 11/37 BLOCO B LOJA 9 COMERCIO LOCAL,GUARA I,BRASILIA,DF,71.010-622,"Q QI 4 BLOCO B, S/N, BRASILIA, DF"
1,96626,2018-02-01,***.024.754-**,00394502017209,CENTRO DE ADESTRAMENTO DA MARAMBAIA (CADIM),"R$ 5.220,00",D,HOSPEDAGENS,00150.000691/2017-84.R.08.26/12/2017.31/12/2018.N,Defesa,FILIAL,101-5 - Órgão Público do Poder Executivo Federal,AV RAFAEL LEVY MIRANDA,S/N,,ITACURUCA,MANGARATIBA,RJ,23.860-000,"AV RAFAEL LEVY MIRANDA, S/N, MANGARATIBA, RJ"
2,96627,2018-02-01,***.002.851-**,03696869000100,BIG TRANS COMERCIAL DE ALIMENTOS LTDA,"R$ 545,06",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,"Comércio varejista de mercadorias em geral, co...",MATRIZ,205-4 - Sociedade Anônima Fechada,SHCN EQ 402/403,SN,BLOCO A,ASA NORTE,BRASILIA,DF,70.834-400,"SHCN EQ 402/403, SN, BRASILIA, DF"
3,96628,2018-02-01,***.002.851-**,38030169000167,SUPERMERCADO VAREJAO OBA LTDA,"R$ 204,19",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,********,MATRIZ,206-2 - Sociedade Empresária Limitada,SCLN QUADRA 306 BLOCO D,S/N,LOJA 79,ASA NORTE,BRASILIA,DF,70.310-500,"SCLN QUADRA 306 BLOCO D, S/N, BRASILIA, DF"
4,96629,2018-02-01,***.002.851-**,47508411103559,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO PÃO DE AÇ...,"R$ 203,20",C,GENEROS DE ALIMENTAÇÃO,00140.000735/2017-95.R.08.09/01/2018.31/12/2018.N,"Comércio varejista de mercadorias em geral, co...",FILIAL,204-6 - Sociedade Anônima Aberta,SHCN EQ 508/509,S/N,BLOCO A,PLANO PILOTO,BRASILIA,DF,70.310-500,"SHCN EQ 508/509, S/N, BRASILIA, DF"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16710,113335,2022-03-12,***.136.591-**,17261661005728,OUTBACK STEAKHOUSE RESTAURANTES BRASIL S/A,"179,5",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,Restaurantes e similares,FILIAL,205-4 - Sociedade Anônima Fechada,Q SCES TRECHO 2,CJ 32,: LOJAS A12A4 E 2A5; : PISO NACOES;,ASA SUL,BRASILIA,DF,70.200-002,"Q SCES TRECHO 2, CJ 32, BRASILIA, DF"
16711,113336,2022-03-12,***.136.591-**,39879944000116,BIOMUNDO 105 NORTE COMERCIO DE ALIMENTOS,"54,95",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,Comércio varejista de produtos alimentícios em...,MATRIZ,206-2 - Sociedade Empresária Limitada,ST SHCN CL QUADRA 105 BLOCO D LOJA,54,,ASA NORTE,BRASILIA,DF,70.734-540,"ST SHCN CL QUADRA 105 BLOCO D LOJA, 54, BRASIL..."
16712,113337,2022-04-12,***.136.591-**,47508411094975,COMPANHIA BRASILEIRA DE DISTRIBUIÇÃO,"518,07",C,GENEROS DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,"Comércio varejista de mercadorias em geral, co...",FILIAL,204-6 - Sociedade Anônima Aberta,SHCN EQ 404/405 BL A,SN,,ASA NORTE,BRASILIA,DF,70.310-500,"SHCN EQ 404/405 BL A, SN, BRASILIA, DF"
16713,113338,2022-04-12,***.136.591-**,12550441000146,RESTAURANTE SOHO,"330,52",C,FORNECIMENTO DE ALIMENTAÇÃO,00264.000642/2022-21.R.08.25/11/2022.31/12/2022.N,Restaurantes e similares,MATRIZ,206-2 - Sociedade Empresária Limitada,ST SHI/S QL 10 LT 22,22,PT SUL,LAGO SUL,BRASILIA,DF,71.630-015,"ST SHI/S QL 10 LT 22, 22, BRASILIA, DF"
